В этом уроке применим линейную регрессию на практике – попробуем предсказать стоимость машин и понять, от каких факторов зависит ценообразование на автомобили. Помимо этого узнаем, какие переменные важны для прогнозирования и насколько хорошо полученная модель описывает данные. 

# Задание:
1. Загрузите данные, проверьте правильность, наличие пропущенных значений, типы данных.
2. Создайте новый признак – марку автомобиля (company). Машины каких производителей встречаются в датасете? Далее исправьте названия и проверьте изменения.
3. Оставьте только часть предикторов, после чего посчитайте корреляцию между price и другими переменными.
4. Преобразуйте категориальные переменные с помощью pd.get_dummies(). 
5. Постройте модель с одним предиктором цены – horsepower. Какой процент изменчивости объясняет полученная модель? (\(R^2\))
6. Далее – две модели (со всеми предикторами и со всеми, кроме марок машин). Обратите внимание на изменения в \(R^2\), коэффициентах и их значимости. Какую модель лучше оставить? 
7. Заполните пропуски в результатах.

## Загрузите данные, проверьте типы, пропущенные значения. 

Сколько пропущенных значений встретилось в датасете?

In [132]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [106]:
cars = pd.read_csv('C:/Users/Александр/pj/karpov_courses/Статистика/Множественный регрессионный анализ/cars.csv')
cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [107]:
cars.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

Ответ: `0`

## Сгенерируем первый признак.

Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company). Для этого используйте столбец CarName, разбейте значения ячеек по пробелу и запишите в колонку первый элемент. Например:
```
'audi 100 ls' → 'audi'
```
Сколько всего уникальных марок машины встречается в датасете? Столбец CarName с полным названием машины удалите из датасета, а также car_ID, они не пригодятся для дальнейшего анализа.

In [108]:
cars['CarName'] = cars.CarName.apply(lambda x: x.split(' ')[0])
cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [109]:
cars.CarName.value_counts().count()

28

Ответ: `28`

## Теперь внимательнее посмотрите на уникальные значения company. Часть из них оказалась с ошибками!
```
'maxda' → 'mazda'
'Nissan' → 'nissan'
'porcshce' → 'porsche'
'toyouta' → 'toyota'
'vokswagen' & 'vw' → 'volkswagen'
```
Сколько уникальных производителей осталось в итоге?

In [110]:
cars = cars.query("CarName not in ('Nissan', 'porcshce', 'vw', 'vokswagen', 'toyouta', 'maxda')")
cars.CarName.value_counts().count()

22

Ответ: `22`

## Отлично! Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:
```
'company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower'
```
также не забыв про то, что мы предсказываем – 'price'. 

После этого посчитайте корреляцию между price и другими переменными. Чему равна корреляция между price и horsepower? Ответ округлите до 2 знаков после точки.

In [111]:
cars_2 = cars[['CarName', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'wheelbase', 'carlength', 'carwidth',
            'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio', 'horsepower', 'price']]
cars_2.head()

,CarName,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower,price
0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,13495.0
1,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,16500.0
2,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154,16500.0
3,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102,13950.0
4,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115,17450.0


In [112]:
cars_2.corr()

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.000000,0.874502,0.794294,0.779521,0.581549,0.500375,0.368960,0.603003
carlength,0.874502,1.000000,0.840107,0.877083,0.686610,0.610150,0.559491,0.695764
carwidth,0.794294,0.840107,1.000000,0.868727,0.741670,0.560484,0.656177,0.773887
curbweight,0.779521,0.877083,0.868727,1.000000,0.851462,0.645432,0.753425,0.841843
enginesize,0.581549,0.686610,0.741670,0.851462,1.000000,0.573355,0.804739,0.871825
boreratio,0.500375,0.610150,0.560484,0.645432,0.573355,1.000000,0.561280,0.538311
horsepower,0.368960,0.559491,0.656177,0.753425,0.804739,0.561280,1.000000,0.800908
price,0.603003,0.695764,0.773887,0.841843,0.871825,0.538311,0.800908,1.000000


Ответ: `0.81`

## Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим pd.get_dummies(). Пример использования:
```python
df_dummy = pd.get_dummies(data=cars[[список_столбцов_типа_object]], drop_first = True)
```
Не забудьте присоединить к полученному датасету столбцы с переменными других типов :) Также обратите внимание: мы должны использовать тот же набор колонок, который был на предыдущем шаге!

Сколько колонок теперь имеется в датасете?

In [113]:
cars_2.dtypes

CarName            object
fueltype           object
aspiration         object
carbody            object
drivewheel         object
wheelbase         float64
carlength         float64
carwidth          float64
curbweight          int64
enginetype         object
cylindernumber     object
enginesize          int64
boreratio         float64
horsepower          int64
price             float64
dtype: object

In [114]:
df_dummy = pd.get_dummies(data=cars_2[['CarName', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype', \
                                       'cylindernumber']], drop_first = True)
df_dummy.head()

,CarName_audi,CarName_bmw,CarName_buick,CarName_chevrolet,CarName_dodge,CarName_honda,CarName_isuzu,CarName_jaguar,CarName_mazda,CarName_mercury,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [116]:
cars_new = pd.concat([cars_2, df_dummy], axis=1)
cars_new = cars_new.drop(columns=['CarName', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype', 'cylindernumber'])
cars_new.shape

(197, 49)

Ответ: `49`

## Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.

Какой процент изменчивости объясняет полученная модель? (округлите до целого)

In [144]:
res = smf.ols(formula='price ~ 1 + horsepower', data=cars_new).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.641
Model:                            OLS   Adj. R-squared:                  0.640
Method:                 Least Squares   F-statistic:                     348.9
Date:                Mon, 25 Jul 2022   Prob (F-statistic):           2.63e-45
Time:                        13:03:10   Log-Likelihood:                -1947.5
No. Observations:                 197   AIC:                             3899.
Df Residuals:                     195   BIC:                             3906.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3693.3512    973.270     -3.795      0.0

## Теперь – две модели:

* модель со всеми предикторами
* модель со всеми предикторами, кроме марок машин

Обратите внимание на изменения в R^2 , коэффициентах и их значимости. Какую модель лучше оставить? 

* **Где меньше предикторов, ведь R2 изменился не очень сильно, а часть марок вообще не значима**
* Вообще с одним предиктором – horsepower!
* Где больше предикторов, там R2 самый большой

## Заполните пропуски:

Выбранная модель объясняет примерно ВЫБРАТЬ% дисперсии (окр. до целого). Среди предикторов ВЫБРАТЬ из 27 оказались не значимыми (p > 0.05). Пример интерпретации: при единичном изменении показателя horsepower, цена ВЫБРАТЬ на ВЫБРАТЬ (без округления).